In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.layers import Input
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:

from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten, Input
from keras.models import Model
import numpy as np
from sklearn import metrics
from keras.preprocessing.image import ImageDataGenerator
digits = [0, 1, 3, 8]
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Генерируем изображения для каждой цифры и режима
for digit in digits:
  for i in range(10):
    img = np.zeros([100,100,3],dtype=np.uint8)
    img.fill(255)
    font = cv2.FONT_HERSHEY_SIMPLEX # для текста
    # Выбираем случайное место для размещения цифры
    x = random.randint(0, 60)
    y = random.randint(50, 100)

    coord = (x, y)
      
    # fontScale
    fontScale = 2

    # Line thickness of 2 px
    thickness = 2
      
    # Using cv2.putText() method
    img = cv2.putText(img,str(digit) , coord, font, fontScale, (0, 0, 0), thickness, cv2.LINE_AA)
    cv2.imwrite(f'/content/drive/MyDrive/lab12cv/test/20/{digit}/{digit}_vertical_{str(i)}.png', img)


for digit in digits:
  for i in range(10):
    img = np.zeros([100,100,3],dtype=np.uint8)
    img.fill(255)
    font = cv2.FONT_HERSHEY_SIMPLEX # для текста
    # Выбираем случайное место для размещения цифры
    x = random.randint(0, 60)
    y = random.randint(50, 100)

    coord = (x, y)
      
    # fontScale
    fontScale = 2

    # Line thickness of 2 px
    thickness = 2
    img = cv2.putText(img,str(digit) , coord, font, fontScale, (0, 0, 0), thickness, cv2.LINE_AA)
    (h, w) = img.shape[:2]
    center = (int(w / 2), int(h / 2))
    rotation_matrix = cv2.getRotationMatrix2D(center, -90, 1)
    
    img = cv2.warpAffine(img, rotation_matrix, (w, h))
    cv2.imwrite(f'/content/drive/MyDrive/lab12cv/test/20/{digit}/{digit}_gorizontal_{str(i)}.png', img)



# Xception

## 700/100/20






In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/700/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')


Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [ ]:
# Загрузка предварительно обученной модели Xception
model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)

83683744/83683744 [==============================] - 5s 0us/step


In [ ]:
# last_layer = model.get_layer('avg_pool').output
# x = Flatten(name='flatten')(last_layer)
# out = Dense(4, activation='softmax', name='output_layer')(x)
# custom_resnet_model = Model(inputs=image_input, outputs=out)
# for layer in base_model.layers:
#     layer.trainable = False
x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(4, activation='softmax')(x)
model_ = keras.Model(inputs=model.input, outputs=x)

# Замораживание всех слоев, кроме последнего
# for layer in model.layers:
#     layer.trainable = False
for layer in model_.layers[:-1]:
  layer.trainable = False
model_.layers[-1].trainable

True

In [ ]:
model_.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_.fit(train_generator,  validation_data=val_generator, epochs=3)

Epoch 1/3
44/44 [==============================] - 19s 167ms/step - loss: 0.6851 - accuracy: 0.8871 - val_loss: 0.3669 - val_accuracy: 0.9900
Epoch 2/3
44/44 [==============================] - 7s 151ms/step - loss: 0.2774 - accuracy: 0.9968 - val_loss: 0.1994 - val_accuracy: 0.9950
Epoch 3/3
44/44 [==============================] - 6s 145ms/step - loss: 0.1742 - accuracy: 0.9989 - val_loss: 0.1345 - val_accuracy: 0.9975


In [ ]:

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)

Found 80 images belonging to 4 classes.


In [ ]:
mpg = model_.predict(test_generator, len(test_generator))

80/80 [==============================] - 3s 11ms/step


In [ ]:
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])



In [ ]:
cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]


In [ ]:
y_t

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '8', '8', '8', '8', '8',
       '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8',
       '8', '8'], dtype='<U1')

In [ ]:
y_p

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',
       '3', '3', '3', '3', '3', '3', '3', '3', '8', '8', '8', '8', '8',
       '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8', '8',
       '8', '8'], dtype='<U1')

In [ ]:
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_t, y_p))

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [ 0  0  0 20]]


## 400/60/20

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/400/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/60/', target_size=(100, 100), batch_size=64, class_mode='categorical')

# Загрузка предварительно обученной модели Xception
model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)

x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(4, activation='softmax')(x)
model_ = keras.Model(inputs=model.input, outputs=x)

# Замораживание всех слоев, кроме последнего
# for layer in model.layers:
#     layer.trainable = False
for layer in model_.layers[:-1]:
  layer.trainable = False
model_.layers[-1].trainable

model_.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_.fit(train_generator,  validation_data=val_generator, epochs=10)
test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model_.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1600 images belonging to 4 classes.
Found 240 images belonging to 4 classes.
Epoch 1/10
25/25 [==============================] - 7s 166ms/step - loss: 0.8745 - accuracy: 0.7719 - val_loss: 0.5581 - val_accuracy: 0.9792
Epoch 2/10
25/25 [==============================] - 3s 130ms/step - loss: 0.4192 - accuracy: 0.9937 - val_loss: 0.3265 - val_accuracy: 0.9875
Epoch 3/10
25/25 [==============================] - 4s 150ms/step - loss: 0.2703 - accuracy: 0.9962 - val_loss: 0.2276 - val_accuracy: 0.9917
Epoch 4/10
25/25 [==============================] - 3s 133ms/step - loss: 0.1982 - accuracy: 0.9975 - val_loss: 0.1759 - val_accuracy: 0.9917
Epoch 5/10
25/25 [==============================] - 3s 135ms/step - loss: 0.1569 - accuracy: 0.9981 - val_loss: 0.1428 - val_accuracy: 0.9958
Epoch 6/10
25/25 [==============================] - 6s 235ms/step - loss: 0.1292 - accuracy: 0.9987 - val_loss: 0.1200 - val_accuracy: 0.9958
Epoch 7/10
25/25 [==============================] - 4s 152ms/step

#№ 250/30/20


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/250/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/30/', target_size=(100, 100), batch_size=64, class_mode='categorical')

# Загрузка предварительно обученной модели Xception
model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)

x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(4, activation='softmax')(x)
model_ = keras.Model(inputs=model.input, outputs=x)

# Замораживание всех слоев, кроме последнего
# for layer in model.layers:
#     layer.trainable = False
for layer in model_.layers[:-1]:
  layer.trainable = False
model_.layers[-1].trainable

model_.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_.fit(train_generator,  validation_data=val_generator, epochs=10)
test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model_.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1000 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Epoch 1/10
16/16 [==============================] - 9s 283ms/step - loss: 1.0547 - accuracy: 0.6320 - val_loss: 0.7517 - val_accuracy: 0.9500
Epoch 2/10
16/16 [==============================] - 2s 139ms/step - loss: 0.6235 - accuracy: 0.9720 - val_loss: 0.4796 - val_accuracy: 1.0000
Epoch 3/10
16/16 [==============================] - 2s 133ms/step - loss: 0.4225 - accuracy: 0.9870 - val_loss: 0.3459 - val_accuracy: 1.0000
Epoch 4/10
16/16 [==============================] - 2s 139ms/step - loss: 0.3191 - accuracy: 0.9890 - val_loss: 0.2685 - val_accuracy: 1.0000
Epoch 5/10
16/16 [==============================] - 2s 143ms/step - loss: 0.2564 - accuracy: 0.9900 - val_loss: 0.2185 - val_accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 2s 137ms/step - loss: 0.2144 - accuracy: 0.9930 - val_loss: 0.1842 - val_accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 2s 149ms/step

## 100/15/20

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/15/', target_size=(100, 100), batch_size=64, class_mode='categorical')

# Загрузка предварительно обученной модели Xception
model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)

x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(4, activation='softmax')(x)
model_ = keras.Model(inputs=model.input, outputs=x)

# Замораживание всех слоев, кроме последнего
# for layer in model.layers:
#     layer.trainable = False
for layer in model_.layers[:-1]:
  layer.trainable = False
model_.layers[-1].trainable

model_.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_.fit(train_generator,  validation_data=val_generator, epochs=3)
test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model_.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 400 images belonging to 4 classes.
Found 60 images belonging to 4 classes.
Epoch 1/3
7/7 [==============================] - 5s 370ms/step - loss: 1.1945 - accuracy: 0.5400 - val_loss: 0.9848 - val_accuracy: 0.7667
Epoch 2/3
7/7 [==============================] - 1s 124ms/step - loss: 0.9298 - accuracy: 0.8650 - val_loss: 0.7579 - val_accuracy: 0.9667
Epoch 3/3
7/7 [==============================] - 1s 132ms/step - loss: 0.7235 - accuracy: 0.9950 - val_loss: 0.5984 - val_accuracy: 0.9667
Found 80 images belonging to 4 classes.
80/80 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       0.95      1.00      0.98        20
           3       1.00      0.95      0.97        20
           8       1.00      0.95      0.97        20

    accuracy                           0.97        80
   macro avg       0.98      0.98      0.97        80
weighted avg       0.98     

# ResNet

#№ 700/100/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/700/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = ResNet152V2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
234545216/234545216 [==============================] - 12s 0us/step


<ipython-input-34-aec75c70843b>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
44/44 [==============================] - 30s 382ms/step - loss: 0.4729 - accuracy: 0.9364 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/3
44/44 [==============================] - 10s 237ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 1.6612e-06 - val_accuracy: 1.0000
Epoch 3/3
44/44 [==============================] - 9s 208ms/step - loss: 8.5611e-04 - accuracy: 0.9996 - val_loss: 1.4645e-05 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 6s 25ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0

## 400/60/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/400/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/60/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = ResNet152V2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=10,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1600 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


<ipython-input-35-5bd7509c8b2d>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=10,validation_data=val_generator)


Epoch 1/10
25/25 [==============================] - 16s 255ms/step - loss: 0.4887 - accuracy: 0.9225 - val_loss: 0.0060 - val_accuracy: 0.9958
Epoch 2/10
25/25 [==============================] - 4s 167ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 1.2620e-06 - val_accuracy: 1.0000
Epoch 3/10
25/25 [==============================] - 4s 145ms/step - loss: 0.0015 - accuracy: 0.9987 - val_loss: 2.9385e-06 - val_accuracy: 1.0000
Epoch 4/10
25/25 [==============================] - 4s 147ms/step - loss: 4.3811e-04 - accuracy: 1.0000 - val_loss: 9.8138e-07 - val_accuracy: 1.0000
Epoch 5/10
25/25 [==============================] - 4s 155ms/step - loss: 3.2918e-05 - accuracy: 1.0000 - val_loss: 4.9916e-07 - val_accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 4s 148ms/step - loss: 2.1788e-05 - accuracy: 1.0000 - val_loss: 3.8493e-07 - val_accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 4s 147ms/step - loss: 0.0011 - accuracy: 0.9994 - val_loss: 1.4

#№ 250/30/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/250/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/30/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = ResNet152V2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1000 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


<ipython-input-36-e2fac2dda3ec>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
16/16 [==============================] - 23s 673ms/step - loss: 0.7672 - accuracy: 0.8720 - val_loss: 2.6247e-05 - val_accuracy: 1.0000
Epoch 2/3
16/16 [==============================] - 3s 182ms/step - loss: 0.0414 - accuracy: 0.9940 - val_loss: 5.9604e-08 - val_accuracy: 1.0000
Epoch 3/3
16/16 [==============================] - 3s 181ms/step - loss: 0.0175 - accuracy: 0.9970 - val_loss: 3.1789e-08 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 8s 33ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0 

## 100/15/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/15/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = ResNet152V2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 400 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


<ipython-input-37-7475c6ff41b0>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
7/7 [==============================] - 22s 997ms/step - loss: 1.8594 - accuracy: 0.7225 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 2/3
7/7 [==============================] - 1s 155ms/step - loss: 0.0548 - accuracy: 0.9775 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 3/3
7/7 [==============================] - 1s 153ms/step - loss: 0.0256 - accuracy: 0.9875 - val_loss: 1.2889e-04 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 6s 28ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [ 0

# InceptionResNetV2

#№ 700/100/200

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/700/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
219055592/219055592 [==============================] - 11s 0us/step


<ipython-input-38-5ec9112fe5eb>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
44/44 [==============================] - 27s 279ms/step - loss: 0.2371 - accuracy: 0.9286 - val_loss: 0.0688 - val_accuracy: 0.9900
Epoch 2/3
44/44 [==============================] - 7s 160ms/step - loss: 0.0619 - accuracy: 0.9854 - val_loss: 0.0485 - val_accuracy: 0.9875
Epoch 3/3
44/44 [==============================] - 9s 189ms/step - loss: 0.0437 - accuracy: 0.9900 - val_loss: 0.0342 - val_accuracy: 0.9925
Found 80 images belonging to 4 classes.
80/80 [==============================] - 8s 41ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [

## 400/60/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/400/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/60/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1600 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


<ipython-input-39-bfc8d667681a>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
25/25 [==============================] - 28s 448ms/step - loss: 0.3103 - accuracy: 0.9019 - val_loss: 0.0952 - val_accuracy: 0.9833
Epoch 2/3
25/25 [==============================] - 4s 155ms/step - loss: 0.0781 - accuracy: 0.9831 - val_loss: 0.0600 - val_accuracy: 0.9875
Epoch 3/3
25/25 [==============================] - 4s 157ms/step - loss: 0.0520 - accuracy: 0.9862 - val_loss: 0.0508 - val_accuracy: 0.9833
Found 80 images belonging to 4 classes.
80/80 [==============================] - 8s 33ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [

## 250/30/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/250/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/30/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1000 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


<ipython-input-40-ce59e449dc60>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
16/16 [==============================] - 29s 574ms/step - loss: 0.4190 - accuracy: 0.8650 - val_loss: 0.1134 - val_accuracy: 0.9917
Epoch 2/3
16/16 [==============================] - 2s 131ms/step - loss: 0.1032 - accuracy: 0.9780 - val_loss: 0.0685 - val_accuracy: 1.0000
Epoch 3/3
16/16 [==============================] - 2s 134ms/step - loss: 0.0734 - accuracy: 0.9900 - val_loss: 0.0554 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 6s 29ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [

## 100/15/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/15/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 400 images belonging to 4 classes.
Found 60 images belonging to 4 classes.
Epoch 1/3


<ipython-input-41-a409bb69c46f>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


7/7 [==============================] - 25s 2s/step - loss: 0.7522 - accuracy: 0.7475 - val_loss: 0.2912 - val_accuracy: 0.9500
Epoch 2/3
7/7 [==============================] - 1s 151ms/step - loss: 0.2018 - accuracy: 0.9550 - val_loss: 0.1987 - val_accuracy: 0.9500
Epoch 3/3
7/7 [==============================] - 1s 150ms/step - loss: 0.1485 - accuracy: 0.9675 - val_loss: 0.1577 - val_accuracy: 0.9500
Found 80 images belonging to 4 classes.
80/80 [==============================] - 10s 42ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [ 0  0  0 20]]


# DenseNet201

## 700/100/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/700/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = DenseNet201(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
74836368/74836368 [==============================] - 4s 0us/step


<ipython-input-42-201ce8ab2cdc>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
44/44 [==============================] - 32s 344ms/step - loss: 0.1421 - accuracy: 0.9654 - val_loss: 6.2585e-09 - val_accuracy: 1.0000
Epoch 2/3
44/44 [==============================] - 7s 164ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 1.3113e-08 - val_accuracy: 1.0000
Epoch 3/3
44/44 [==============================] - 7s 157ms/step - loss: 0.0014 - accuracy: 0.9993 - val_loss: 2.6822e-09 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 6s 19ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0 

## 400/60/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/400/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/60/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = DenseNet201(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1600 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


<ipython-input-43-ed30a322884f>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
25/25 [==============================] - 25s 360ms/step - loss: 0.1977 - accuracy: 0.9362 - val_loss: 3.1767e-06 - val_accuracy: 1.0000
Epoch 2/3
25/25 [==============================] - 3s 136ms/step - loss: 0.0015 - accuracy: 0.9987 - val_loss: 2.4835e-09 - val_accuracy: 1.0000
Epoch 3/3
25/25 [==============================] - 3s 136ms/step - loss: 2.1475e-04 - accuracy: 1.0000 - val_loss: 2.9802e-09 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 6s 22ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 

#№ 250/30/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/250/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/30/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = DenseNet201(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1000 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


<ipython-input-44-c5f7f3582344>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
16/16 [==============================] - 32s 962ms/step - loss: 0.4024 - accuracy: 0.8820 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 2/3
16/16 [==============================] - 3s 170ms/step - loss: 0.0044 - accuracy: 0.9970 - val_loss: 2.9802e-09 - val_accuracy: 1.0000
Epoch 3/3
16/16 [==============================] - 3s 201ms/step - loss: 0.0023 - accuracy: 0.9990 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 10s 44ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 

## 100/15/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/15/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = DenseNet201(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 400 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


<ipython-input-45-f7d53e2a4c09>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
7/7 [==============================] - 22s 1s/step - loss: 2.1065 - accuracy: 0.6200 - val_loss: 0.0236 - val_accuracy: 1.0000
Epoch 2/3
7/7 [==============================] - 1s 183ms/step - loss: 0.0819 - accuracy: 0.9775 - val_loss: 1.3259e-05 - val_accuracy: 1.0000
Epoch 3/3
7/7 [==============================] - 1s 135ms/step - loss: 0.0337 - accuracy: 0.9925 - val_loss: 2.2444e-05 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 4s 20ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      0.95      0.97        20
           3       0.95      1.00      0.98        20
           8       1.00      1.00      1.00        20

    accuracy                           0.99        80
   macro avg       0.99      0.99      0.99        80
weighted avg       0.99      0.99      0.99        80

[[20  0  0  0]
 [ 0 19  1  0]
 [ 0  0 20  0]
 [ 

# NASNetLarge

## 700/100/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/700/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = NASNetLarge(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 2800 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
343610240/343610240 [==============================] - 16s 0us/step


<ipython-input-46-ddb18fe269cb>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
44/44 [==============================] - 61s 689ms/step - loss: 0.1498 - accuracy: 0.9529 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/3
44/44 [==============================] - 12s 278ms/step - loss: 0.0108 - accuracy: 0.9979 - val_loss: 6.9373e-04 - val_accuracy: 1.0000
Epoch 3/3
44/44 [==============================] - 12s 272ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 4.3462e-04 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 14s 59ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  

## 400/60/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/400/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/60/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = NASNetLarge(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1600 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


<ipython-input-47-2b934c077d15>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
25/25 [==============================] - 31s 493ms/step - loss: 0.2053 - accuracy: 0.9294 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 2/3
25/25 [==============================] - 6s 250ms/step - loss: 0.0057 - accuracy: 0.9994 - val_loss: 8.3480e-04 - val_accuracy: 1.0000
Epoch 3/3
25/25 [==============================] - 6s 253ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.7353e-04 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 8s 30ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 2

## 250/30/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/250/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/30/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = NASNetLarge(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 1000 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


<ipython-input-48-b15e1157b526>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
16/16 [==============================] - 29s 709ms/step - loss: 0.4010 - accuracy: 0.8570 - val_loss: 0.0158 - val_accuracy: 1.0000
Epoch 2/3
16/16 [==============================] - 4s 255ms/step - loss: 0.0149 - accuracy: 0.9980 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 3/3
16/16 [==============================] - 4s 251ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 8s 34ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [

## 100/15/20

In [ ]:
img_width, img_height = 100, 100

input_shape = (img_width, img_height, 3) # 3 - channels_last
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 100
# Количество изображений для проверки
nb_validation_samples = 14
# Количество изображений для тестирования
nb_test_samples = 20

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/train/100/', target_size=(100, 100), batch_size=64, class_mode='categorical')
val_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/valid/15/', target_size=(100, 100), batch_size=64, class_mode='categorical')


vgg16_net = NASNetLarge(weights='imagenet',include_top=False,input_shape=(100, 100, 3))
vgg16_net.trainable = False

model = Sequential()
model.add(vgg16_net) 
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit_generator(train_generator,epochs=3,validation_data=val_generator)

test_generator = datagen.flow_from_directory('/content/drive/MyDrive/lab12cv/test/20/', target_size=(100, 100), batch_size=1, class_mode='categorical', shuffle=False)
mpg = model.predict(test_generator, len(test_generator))
y_test = test_generator.classes
y_pred = np.array([np.argmax(mpg[i,:]) for i in range(mpg.shape[0])])

cl_names = list(train_generator.class_indices)
y_t = np.array(cl_names)[y_test]
y_p = np.array(cl_names)[y_pred]
mtrs = metrics.classification_report(y_t, y_p)
print(mtrs)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_t, y_p))

Found 400 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


<ipython-input-49-3e21cf7fd243>:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3,validation_data=val_generator)


Epoch 1/3
7/7 [==============================] - 28s 2s/step - loss: 0.6693 - accuracy: 0.7325 - val_loss: 0.0581 - val_accuracy: 1.0000
Epoch 2/3
7/7 [==============================] - 2s 247ms/step - loss: 0.1008 - accuracy: 0.9775 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 3/3
7/7 [==============================] - 2s 270ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Found 80 images belonging to 4 classes.
80/80 [==============================] - 9s 36ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           8       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

[[20  0  0  0]
 [ 0 20  0  0]
 [ 0  0 20  0]
 [ 0  0  0 